BERT TF script, Refactored:

In [ ]:
# Progress Counter
import sys

class progressCounter():

  def __init__(self, num_iterations, name=''):
    self.progress = 0
    self.N = num_iterations
    self.name = name


  def check_pt(self, i):
    curr_progress = int(i/(self.N-1) * 100)
    if curr_progress - self.progress > 0:
      self.progress = curr_progress
      sys.stdout.write("\r{1} Progress: {0}%".format(self.progress, self.name))
      sys.stdout.flush()

In [ ]:
# Instantiate BERT from Hub
import tensorflow as tf
import tensorflow_hub as hub

max_seq_length = 512  # Your choice here.
input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                       name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                   name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                    name="segment_ids")

bert_inputs = [input_word_ids, input_mask, segment_ids]

bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/2",
                            trainable=True, name='bert_layer')
pooled_output, sequence_output = bert_layer(bert_inputs)


In [ ]:
# Import and Build BERT Tokenizer

!pip install tf-models-official
import official.nlp.bert.tokenization as tokenization
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

import numpy as np
# Convert To BERT Tokens
def convertToBERTInputData(sentence):
  sent_tokenized = tokenizer.tokenize(sentence)
  sent_length = len(sent_tokenized)
  if sent_length > 510:
    sent_tokenized = sent_tokenized[0:510]
    sent_length = 510

  sent_tokenized.insert(0,'[CLS]')
  sent_tokenized.append('[SEP]')
  sent_length+=2

  sent_ids = tokenizer.convert_tokens_to_ids(sent_tokenized) + [0] * (512 - sent_length)
  sent_mask = [1]*sent_length + [0]*(512 - sent_length)
  sent_seg_ids =  [0] * 512

  sent_ids = np.array(sent_ids, dtype=np.int32)
  sent_mask = np.array(sent_mask, dtype=np.int32)
  sent_seg_ids = np.array(sent_seg_ids, dtype=np.int32)

  return ( sent_ids, sent_mask, sent_seg_ids )

     |████████████████████████████████| 849kB 8.4MB/s 
     |████████████████████████████████| 1.1MB 30.0MB/s 
     |████████████████████████████████| 36.6MB 87kB/s 
     |████████████████████████████████| 174kB 50.6MB/s 
     |████████████████████████████████| 102kB 13.1MB/s 
     |████████████████████████████████| 358kB 50.6MB/s 
  Created wheel for py-cpuinfo: filename=py_cpuinfo-7.0.0-cp36-none-any.whl size=20069 sha256=040e0f8d367d67fdf49252c1fd8e7049ba168c73e42eecd1f3eac5ee77eae2e9
  Stored in directory: /root/.cache/pip/wheels/f1/93/7b/127daf0c3a5a49feb2fecd468d508067c733fba5192f726ad1
Successfully built py-cpuinfo


Script Begin:

In [ ]:
from sklearn.datasets import fetch_20newsgroups

newsgroups_train = fetch_20newsgroups(data_home=None, subset='train', categories=None, shuffle=True, random_state=42, remove=(), download_if_missing=True, return_X_y=False)
newsgroups_test = fetch_20newsgroups(data_home=None, subset='test', categories=None, shuffle=True, random_state=42, remove=(), download_if_missing=True, return_X_y=False)

INFO:sklearn.datasets._twenty_newsgroups:Downloading 20news dataset. This may take a few minutes.
INFO:sklearn.datasets._twenty_newsgroups:Downloading dataset from https://ndownloader.figshare.com/files/5975967 (14 MB)


Train Data:

In [ ]:
data = newsgroups_train.data
targets = newsgroups_train.target
categories = newsgroups_train.target_names

In [ ]:
# Create Feature Inputs -- X
data_len = len(data)
pC = progressCounter(data_len)
bert_inputs_x1 = [0]*data_len
bert_inputs_x2 = [0]*data_len
bert_inputs_x3 = [0]*data_len

for i in range(data_len):
  temp = convertToBERTInputData(data[i])
  bert_inputs_x1[i] = temp[0]
  bert_inputs_x2[i] = temp[1]
  bert_inputs_x3[i] = temp[2]
  pC.check_pt(i)


 Progress: 100%

In [ ]:
import numpy as np
# Create Feature Outputs -- Y
data_len = len(targets)
expected_output = [0] * data_len
for i in range(data_len):
  expected_output[i] = [0]*20

for i in range(data_len):
  index = targets[i]
  expected_output[i][index] = 1
  expected_output[i] = np.array(expected_output[i], dtype=np.int32)

expected_output_y = expected_output # <----------- y value

Create TF -- BERT Model:

In [ ]:
from official import nlp

classifier_layer_1 = tf.keras.layers.Dense(512, name='classification_layer_1',activation='relu')(pooled_output)
classifier_layer_2 = tf.keras.layers.Dense(256, name='classification_layer_2',activation='relu')(classifier_layer_1)
predictions = tf.keras.layers.Dense(20, name='classification_layer_3',activation='sigmoid')(classifier_layer_2)

model = tf.keras.models.Model(inputs=bert_inputs, outputs=predictions, name='Bert_Tensorflow')
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics='categorical_accuracy') #add from logits


In [ ]:
model.summary()

Model: "Bert_Tensorflow"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 512)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 512)]        0                                            
__________________________________________________________________________________________________
bert_layer (KerasLayer)         [(None, 1024), (None 335141889   input_word_ids[0][0]             
                                                                 input_mask[0][0]   

In [ ]:
expected_output_y = np.array(expected_output_y, dtype = np.int32)
expected_output_y = np.reshape(expected_output_y,(11314,20))
print(expected_output_y.shape)

bert_inputs_x1 = np.array(bert_inputs_x1,dtype=np.int32)
bert_inputs_x1 = np.reshape(bert_inputs_x1,(11314,512))

bert_inputs_x2 = np.array(bert_inputs_x2,dtype=np.int32)
bert_inputs_x2 = np.reshape(bert_inputs_x2,(11314,512))

bert_inputs_x3 = np.array(bert_inputs_x3,dtype=np.int32)
bert_inputs_x3 = np.reshape(bert_inputs_x3,(11314,512))

print(bert_inputs_x1.shape)

(11314, 20)
(11314, 512)


In [ ]:
model.fit({'input_word_ids' : bert_inputs_x1, 'input_mask' : bert_inputs_x2 , 'segment_ids' : bert_inputs_x3 }, {'classification_layer_3' : expected_output_y}, verbose=1, batch_size=1)

 3782/11314 [=========>....................] - ETA: 34:27 - loss: 2.9922 - categorical_accuracy: 0.0539